In [1]:
!pip install streamlit -q

In [ ]:
pip install 

In [2]:
%%writefile app.py

UsageError: %%writefile is a cell magic, but the cell body is empty.


In [5]:
import numpy as np
import streamlit as st
from PIL import Image
from sklearn.cluster import KMeans
from collections import Counter
import tensorflow as tf
import webcolors

# Load pre-trained MobileNetV2 model for feature extraction
model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False

# Predefined list of color names
color_names = list(webcolors.CSS3_NAMES_TO_HEX.keys())

def preprocess_image(image):
    # Resize and preprocess image
    resized_image = image.resize((224, 224))
    img_array = np.array(resized_image)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

def extract_features(image, model):
    img_array = preprocess_image(image)
    features = model.predict(img_array)
    return features

def closest_color(requested_color):
    min_colors = {}
    for color_name, color_code in webcolors.CSS3_NAMES_TO_HEX.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(color_code)
        rd = (r_c - int(requested_color[1:3], 16)) ** 2
        gd = (g_c - int(requested_color[3:5], 16)) ** 2
        bd = (b_c - int(requested_color[5:7], 16)) ** 2
        min_colors[(rd + gd + bd)] = color_name
    return min_colors[min(min_colors.keys())]

def detect_color_with_features(image, model):
    # Resize image maintaining the aspect ratio
    resized_image = image.resize((224, 224))
    img_array = np.array(resized_image) / 255.0

    # Reshape the image array to 2D (flattened)
    img_array_reshaped = img_array.reshape(-1, 3)

    # Determine the optimal number of clusters using the elbow method
    distortions = []
    K = range(1, 11)
    for k in K:
        kmeans = KMeans(n_clusters=k)
        kmeans.fit(img_array_reshaped)
        distortions.append(kmeans.inertia_)

    # Determine the optimal number of clusters based on the elbow point
    optimal_k = np.argmin(np.diff(distortions)) + 2

    # Apply k-means clustering with the optimal number of clusters
    kmeans = KMeans(n_clusters=optimal_k)
    kmeans.fit(img_array_reshaped)

    # Get cluster centers (representative colors)
    cluster_centers = kmeans.cluster_centers_

    # Convert RGB values to hexadecimal color codes
    hex_colors = ['#' + ''.join(f'{int(c*255):02x}' for c in color) for color in cluster_centers]

    # Get closest color name for each detected color
    detected_color_names = []
    for hex_code in hex_colors:
        detected_color_name = closest_color(hex_code)
        detected_color_names.append(detected_color_name)

    # Calculate percentage of each color in the image
    labels = kmeans.labels_
    counts = Counter(labels)
    total_pixels = len(labels)
    percentages = [(count / total_pixels) * 100 for count in counts.values()]

    # Combine color names, hex codes, and percentages
    detected_colors = [(color_name, hex_code, percentage) for color_name, hex_code, percentage in zip(detected_color_names, hex_colors, percentages)]

    return detected_colors

# Streamlit app setup
st.set_page_config(layout="wide")

hide_streamlit_style = """
            <style>
            #MainMenu {visibility: hidden;}
            footer {visibility: hidden;}
            </style>
            """
st.markdown(hide_streamlit_style, unsafe_allow_html=True)

if __name__ == "__main__":
    col1 = st.sidebar

    col1.markdown('# Upload image for color detection')
    uploaded_file = col1.file_uploader("Choose image")
    if uploaded_file is not None:
        content_image = Image.open(uploaded_file)
        col1.image(content_image, caption='Uploaded Image', use_column_width=True)
        if st.button("Detect Color"):
            detected_colors = detect_color_with_features(content_image, model)
            st.write("Detected Colors:")
            for color_name, hex_code, percentage in detected_colors:
                st.write(f"Color: {color_name} - Hex: {hex_code} - Percentage: {percentage:.2f}%")


ModuleNotFoundError: No module named 'distutils'

In [ ]:
!wget -q -O - ipv4.icanhazip.com

35.194.200.203


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.200.203:8501

npx: installed 22 in 3.242s
your url is: https://common-swans-crash.loca.lt
2024-05-18 12:59:13.959229: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-18 12:59:13.959312: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-18 12:59:13.961221: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-18 12:59:15.816335: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/usr/local/lib/python3.